In [8]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from rainpy.raindrop import make_frame

# from collections import namedtuple
# drops = {"name": "example", 
#                 "floder": "./example", 
#                 "author": "zmdsn",
#                 "email":"zmdsn@126.com",
#                 "url":"https://github.com/zmdsn/rainpy"}
# RainDrop = namedtuple("RainDrop", drops.keys())
# drop = RainDrop(**drops)

# make_frame

In [ ]:
# import pandas as pd
# 读取 CSV 文件（假设文件名为 `data.csv`，且包含表头）
# df = pd.read_csv('../glory/predict/datasets/ETT-small/ETTh1.csv')
# df.describe().to_json()
# df['date']


0        2016-07-01 00:00:00
1        2016-07-01 01:00:00
2        2016-07-01 02:00:00
3        2016-07-01 03:00:00
4        2016-07-01 04:00:00
                ...         
17415    2018-06-26 15:00:00
17416    2018-06-26 16:00:00
17417    2018-06-26 17:00:00
17418    2018-06-26 18:00:00
17419    2018-06-26 19:00:00
Name: date, Length: 17420, dtype: object

In [18]:
# df.index.is_uniform()
time_diff = df.index.to_series().diff()
print("时间间隔是否一致:", time_diff.nunique() == 1)
# df.index.to_series().diff()

时间间隔是否一致: True


df.dtypes

In [ ]:
from rainpy import read
import pandas as pd
from rainpy.ts import get_datetime_column, set_datetime_index

df  = read('../glory/predict/datasets/ETT-small/ETTh1.csv')
# get_datetime_column(df)

df = set_datetime_index(df)
# df.index.to_series().diff().describe()
time_diffs = df.index.to_series().diff() # 计算相邻时间差
time_diffs[1:].unique()

/mnt/d/rainpy/src/rainpy/read.py:126: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return function(file, *args, **filter_kwargs)


<TimedeltaArray>
['0 days 01:00:00']
Length: 1, dtype: timedelta64[ns]

In [30]:
# df.shape[0]>2
import pandas as pd
url = "https://example.com/data.csv"
data = pd.read_csv(url)
data.head()

HTTPError: HTTP Error 404: Not Found

In [32]:
def analyze_sampling_quality(df, expected_freq='S'):
    """
    综合分析数据采样质量
    """
    # 计算实际时间间隔
    time_diffs = df.index.to_series().diff().dropna()
    
    # 将期望频率转换为Timedelta
    expected_interval = pd.Timedelta(expected_freq)
    
    print("=== 采样质量分析报告 ===")
    print(f"期望采样间隔: {expected_interval}")
    print(f"实际数据点数量: {len(df)}")
    print(f"时间范围: {df.index.min()} 到 {df.index.max()}")
    
    # 分析1: 时间间隔统计
    print("\n--- 时间间隔分析 ---")
    print(f"平均间隔: {time_diffs.mean()}")
    print(f"间隔标准差: {time_diffs.std()}")
    print(f"最小间隔: {time_diffs.min()}")
    print(f"最大间隔: {time_diffs.max()}")
    
    # 分析2: 缺失数据评估
    expected_count = len(pd.date_range(
        start=df.index.min(), 
        end=df.index.max(), 
        freq=expected_freq
    ))
    missing_ratio = 1 - len(df) / expected_count
    print(f"\n--- 缺失数据评估 ---")
    print(f"预期数据点: {expected_count}")
    print(f"实际数据点: {len(df)}")
    print(f"缺失比例: {missing_ratio:.2%}")
    
    # 分析3: 间隔分布
    print("\n--- 间隔分布分析 ---")
    unique_intervals = time_diffs.value_counts().head(10)
    print("最常见的10个间隔:")
    for interval, count in unique_intervals.items():
        print(f"  {interval}: {count}次 ({count/len(time_diffs):.1%})")
    
    return time_diffs, missing_ratio

analyze_sampling_quality(df)

ValueError: unit abbreviation w/o a number

In [34]:
from pandas.tseries.frequencies import infer_freq
freq = infer_freq(df.index)
print("时间频率:", freq)

时间频率: h


In [ ]:
from rainpy.ts import get_datetime_column
from rainpy import read
import pandas as pd

def set_datetime_index(df):
    datetime_col, ds_type, x = get_datetime_column(df)
    print(datetime_col, ds_type, x)
    if datetime_col:
        df[datetime_col] = pd.to_datetime(df[datetime_col])
        if ds_type in ['s', 'ms', 'us']:
            df[datetime_col] = pd.to_datetime(df[datetime_col], unit=ds_type)
            
        df.set_index(datetime_col, inplace=True)
    return df

In [2]:
import pandas as pd

# 示例数据 - 不同精度的时间戳
timestamps = {
    'seconds': [1609459200, 1609545600, 1609632000],      # 秒级
    'milliseconds': [1609459200000, 1609545600000, 1609632000000],  # 毫秒级
    'microseconds': [1609459200000000, 1609545600000000, 1609632000000000]  # 微秒级
}

df = pd.DataFrame(timestamps)

# # 转换为datetime
df['datetime_seconds'] = pd.to_datetime(df['seconds'], unit='s')
# df['datetime_ms'] = pd.to_datetime(df['milliseconds'], unit='ms')
# df['datetime_us'] = pd.to_datetime(df['microseconds'], unit='us')

# df = set_datetime_index(df)
df

,seconds,milliseconds,microseconds,datetime_seconds
0,1609459200,1609459200000,1609459200000000,2021-01-01
1,1609545600,1609545600000,1609545600000000,2021-01-02
2,1609632000,1609632000000,1609632000000000,2021-01-03


ImportError: cannot import name 'is_time_index_continuous' from 'rainpy.ts' (/mnt/d/rainpy/src/rainpy/ts.py)

In [ ]:
%load_ext autoreload
%autoreload 1
from rainpy.ts import read_data, get_datetime_column
from rainpy import read

file_path="tests/test_data/csv_utf8_t.csv"

df = read(file_path)
# get_datetime_column(df)
from rainpy.ts import is_time_index_continuous

is_time_index_continuous(df)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/mnt/d/rainpy/src/rainpy/read.py:126: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return function(file, *args, **filter_kwargs)


ImportError: cannot import name 'is_time_index_continuous' from 'rainpy.ts' (/mnt/d/rainpy/src/rainpy/ts.py)

In [6]:
data = {
    'category': ['A', 'B', 'A', 'C', 'A', 'B', 'A', 'C', 'A'],
    'values': [1, 2, 2, 3, 2, 1, 2, 3, 2],
    'scores': [85, 90, 85, 92, 85, 90, 88, 92, 85],
    'mixed': ['X', 'Y', 'X', 'Z', 'X', 'Y', 'X', 0, 'X']
}

df = pd.DataFrame(data)

df['category'].mode()[0]  # 返回出现频率最高的值

'A'